# Import Packages

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import RepeatedKFold
from numpy import arange
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split, GridSearchCV

# Data Load & Feature Engineering

In [2]:
#Data Load
bpsell= pd.read_excel(r'D:\Users\91709\Downloads\Work\UpWork\2023-24\Gus\Data\BP SELLOUT Own Brands.xlsx', \
                      sheet_name= 'BP SELLOUT JAN TO SEP 2023- MAS') #BP Sell out Data
bpinfo= pd.read_excel(r'D:\Users\91709\Downloads\Work\UpWork\2023-24\Gus\Data\BP Headcount_MASKED.xlsx') #BP Headcount
outletinfo= pd.read_excel(r'D:\Users\91709\Downloads\Work\UpWork\2023-24\Gus\Data\Outlet Detail.xlsx') #Outlet Feature

# Feature Engineering

In [3]:
#Data Filter
bpsell= bpsell[bpsell['amount (in bottles)'] > 0 ] #removing records that has 0 or negative amounts in bottles
bpsell= bpsell[bpsell['amount (in bottles)'].notnull()] #removing records that has null value

#Feature Engineering
bpinfo['Age'] = ((pd.to_datetime('2023-09-30') - bpinfo['Date of Birth']).dt.days) / 365 #Extracted Age from Date of Birth
bpinfo['Tenure'] = ((pd.to_datetime('2023-09-30') - bpinfo['Hire Date']).dt.days) / 365 #Extracted Tenure from Hire Date

#Column Selection
bpinfo= bpinfo[['Global Personnel ID', 'Nationality', 'Age', 'Tenure', 'Salary']]

#Data Merge
bpinfo_merge= bpsell.merge(bpinfo, how= 'inner', left_on= 'bp_id', right_on= 'Global Personnel ID') #Merge BP Headcount with
#BP Sell out on BP ID (should be available in both data)
data= bpinfo_merge.merge(outletinfo, how= 'inner', left_on= 'outlet_id', right_on= 'Outlet #') #Merge the above merged data 
#with outlet feature on outlet id (should be available on both data)

#Date Manipulation & Feature Engineering
data['Date'] = data['recorded_on'].str[:10] #Extracted date from Date time format of recorded on
data['Date'] = pd.to_datetime(data['Date']) #Converted the date column to date format from object

data['month_name'] = data['Date'].dt.month_name() #Extracted Month from Date column
data['day_of_week'] = data['Date'].dt.day_name() #Extracted Day of the week from Date column

#Drop columns that are not required
data = data.drop(['recorded_on', 'created_on','outlet_id', 'Global Personnel ID', 'Outlet #', 'Date'], axis=1)

#Data Binning
age_bin= [18, 25, 35, 45, 55, 65, float('inf') ] #Created the class interval for age
age_labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+'] #Labelled the class interval for age
data['Age_Bins'] = pd.cut(data['Age'], bins=age_bin, labels=age_labels) #Binned age

tenure_bin= [-1, 3, 6, 11, 15, float('inf')] #Created the class interval for tenure
tenure_labels = ['0-2', '3-5', '6-10', '11-15', '15+'] #Labelled the class interval for tenure
data['Tenure_Bins'] = pd.cut(data['Tenure'], bins=tenure_bin, labels=tenure_labels) #Binned tenure

salary_bin= [1000, 1201, 1401, 1601, 1801, float('inf')] #Created the class interval for salary
salary_labels = ['1000-1200', '1201-1400', '1401-1600', '1601-1800', '1800+'] #Labelled the class interval for salary
data['Salary_Bins'] = pd.cut(data['Salary'], bins=salary_bin, labels=salary_labels) #Binned salary

#Drop columns that are not required
data_transform = data.drop(['Age', 'Tenure','Salary', 'bp_id'], axis=1)

#One Hot Encoding
data_transform = pd.get_dummies(data_transform, columns=['brand name'], prefix='Brand', dtype=float) #Encoded Brand
data_transform = pd.get_dummies(data_transform, columns=['Nationality'], prefix='Nationality', dtype=float) #Encoded Nationality
data_transform = pd.get_dummies(data_transform, columns=['Outlet Type'], prefix='Outlet Type', dtype=float) #Outlet Type
data_transform = pd.get_dummies(data_transform, columns=['Outlet Group'], prefix='Outlet Group', dtype=float) #Outlet Group
data_transform = pd.get_dummies(data_transform, columns=['month_name'], prefix='Month', dtype=float) #Month
data_transform = pd.get_dummies(data_transform, columns=['day_of_week'], prefix='Day', dtype=float) #Day

data_transform = pd.get_dummies(data_transform, columns=['Salary_Bins'], prefix='Salary', dtype=float) #Salary
data_transform = pd.get_dummies(data_transform, columns=['Age_Bins'], prefix='Age', dtype=float) #Age
data_transform = pd.get_dummies(data_transform, columns=['Tenure_Bins'], prefix='Tenure', dtype=float) #Tenure

data_transform

,amount (in bottles),Brand_Crystal,Brand_Guinness,Brand_Heineken,Brand_Tiger,Nationality_Cambodian,Nationality_Chinese,Nationality_Indonesian,Nationality_Malaysian,Nationality_Singaporean,...,Age_25-34,Age_35-44,Age_45-54,Age_55-64,Age_65+,Tenure_0-2,Tenure_3-5,Tenure_6-10,Tenure_11-15,Tenure_15+
0,13.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,69.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,36.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,13.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131308,38.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
131309,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
131310,60.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
131311,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# Machine Learning Models

In [4]:
#Splitting independent and dependent variables
X = data_transform.drop('amount (in bottles)', axis=1) 
y = data_transform['amount (in bottles)']

Lasso CV

In [5]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

reg = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
LassoCV()

reg.fit(X, y)

print('Best alpha using built-in LassoCV: %f' % reg.alpha_)
print('R2: %f' % reg.score(X,y))
print('Adjusted R2: %f' % (1 - (1-reg.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)))
y_predicted = reg.predict(X)
print('RMSE: %f' % (mean_squared_error(y, y_predicted, squared=False)))
print('y intercept of the model: %f' % reg.intercept_)
coef = pd.Series(reg.coef_, index = X.columns)
print('Lasso picked ' + str(sum(coef != 0)) + ' variables and eliminated the other ' + str(sum(coef == 0)) + ' variables')
coef = coef[coef !=0]
imp_coef = coef.sort_values(ascending = False)
print(imp_coef)

Best alpha using built-in LassoCV: 0.000000
R2: 0.355952
Adjusted R2: 0.355475
RMSE: 35.884313
y intercept of the model: 58.595743
Lasso picked 97 variables and eliminated the other 0 variables
Outlet Type_FAMILY FOOD COURT             757.444843
Outlet Group_YUNG CHING RD                 18.710468
Outlet Group_BUKIT PANJANG                 17.893276
Age_18-24                                  16.889948
Nationality_Indonesian                     16.658709
                                             ...    
Outlet Type_COFFEE SHOPS - NON-BP         -17.248535
Outlet Type_COFFEE SHOPS - BP APBS        -19.297037
Outlet Type_COFFEE SHOPS - BP NON-APBS    -29.179477
Brand_Crystal                             -30.455410
Brand_Guinness                            -32.305526
Length: 97, dtype: float64


Poisson Regressor

In [6]:
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
}

poisson_best_param = PoissonRegressor()

grid_search = GridSearchCV(poisson_best_param, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

best_alpha = grid_search.best_params_['alpha']

poisson_model = PoissonRegressor(alpha=best_alpha)
poisson_model.fit(X, y)

predictions = poisson_model.predict(X)

print('R2: %f' % poisson_model.score(X,y))
print('Adjusted R2: %f' % (1 - (1-poisson_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)))
print('RMSE: %f' % (mean_squared_error(y, predictions, squared=False)))
print('y intercept of the model: %f' % poisson_model.intercept_)

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': poisson_model.coef_})
coefficients

R2: 0.555434
Adjusted R2: 0.555106
RMSE: 38.026977
y intercept of the model: 3.476855


,Feature,Coefficient
0,Brand_Crystal,-0.620002
1,Brand_Guinness,-0.869911
2,Brand_Heineken,0.751394
3,Brand_Tiger,0.738558
4,Nationality_Cambodian,0.013235
...,...,...
92,Tenure_0-2,0.060860
93,Tenure_3-5,0.075682
94,Tenure_6-10,-0.018771
95,Tenure_11-15,-0.077388


Genralize linear model power = 0

In [7]:
glm_model = TweedieRegressor(power=0, link='log')

glm_model.fit(X, y)
predictions = glm_model.predict(X)

print('R2: %f' % glm_model.score(X,y))
print('Adjusted R2: %f' % (1 - (1-glm_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)))
print('RMSE: %f' % (mean_squared_error(y, predictions, squared=False)))

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': glm_model.coef_})
print(coefficients)

R2: 0.383583
Adjusted R2: 0.383128
RMSE: 35.106093
                  Feature  Coefficient
0           Brand_Crystal    -0.641769
1          Brand_Guinness    -1.003418
2          Brand_Heineken     0.862971
3             Brand_Tiger     0.830950
4   Nationality_Cambodian     0.107428
..                    ...          ...
92             Tenure_0-2     0.116694
93             Tenure_3-5     0.136394
94            Tenure_6-10    -0.034511
95           Tenure_11-15    -0.100046
96             Tenure_15+    -0.069797

[97 rows x 2 columns]


Genralize linear model power = 1

In [8]:
glm_model = TweedieRegressor(power=1, link='log')

glm_model.fit(X, y)
predictions = glm_model.predict(X)

print('R2: %f' % glm_model.score(X,y))
print('Adjusted R2: %f' % (1 - (1-glm_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)))
print('RMSE: %f' % (mean_squared_error(y, predictions, squared=False)))

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': glm_model.coef_})
print(coefficients)

R2: 0.523189
Adjusted R2: 0.522837
RMSE: 39.160006
                  Feature  Coefficient
0           Brand_Crystal    -0.558073
1          Brand_Guinness    -0.771645
2          Brand_Heineken     0.668961
3             Brand_Tiger     0.660712
4   Nationality_Cambodian     0.004244
..                    ...          ...
92             Tenure_0-2     0.046428
93             Tenure_3-5     0.070989
94            Tenure_6-10    -0.011624
95           Tenure_11-15    -0.067760
96             Tenure_15+    -0.038077

[97 rows x 2 columns]


Elastic Net

In [9]:
elastic_net_model = ElasticNet(alpha=0.001, l1_ratio=1)

elastic_net_model.fit(X, y)
predictions = elastic_net_model.predict(X)

print('R2: %f' % elastic_net_model.score(X,y))
print('Adjusted R2: %f' % (1 - (1-elastic_net_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)))
print('RMSE: %f' % (mean_squared_error(y, predictions, squared=False)))

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': elastic_net_model.coef_})
print(coefficients)

R2: 0.355946
Adjusted R2: 0.355470
RMSE: 35.884465
                  Feature  Coefficient
0           Brand_Crystal   -30.446572
1          Brand_Guinness   -32.299370
2          Brand_Heineken     5.192301
3             Brand_Tiger     3.585270
4   Nationality_Cambodian     0.740496
..                    ...          ...
92             Tenure_0-2     3.136471
93             Tenure_3-5     2.622051
94            Tenure_6-10    -1.169690
95           Tenure_11-15    -2.206430
96             Tenure_15+    -0.000000

[97 rows x 2 columns]


Baysein Ridge

In [10]:
baysein_model = BayesianRidge()

baysein_model.fit(X, y)
predictions = baysein_model.predict(X)

print('R2: %f' % baysein_model.score(X,y))
print('Adjusted R2: %f' % (1 - (1-baysein_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)))
print('RMSE: %f' % (mean_squared_error(y, predictions, squared=False)))

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': baysein_model.coef_})
print(coefficients)

R2: 0.355950
Adjusted R2: 0.355474
RMSE: 35.884347
                  Feature  Coefficient
0           Brand_Crystal   -16.961068
1          Brand_Guinness   -18.810867
2          Brand_Heineken    18.689643
3             Brand_Tiger    17.082292
4   Nationality_Cambodian    -0.334488
..                    ...          ...
92             Tenure_0-2     2.667282
93             Tenure_3-5     2.163373
94            Tenure_6-10    -1.659222
95           Tenure_11-15    -2.693061
96             Tenure_15+    -0.478372

[97 rows x 2 columns]


Run the models again after removing 2.5% outliers from top and bottom

In [11]:
lower_bound = data_transform['amount (in bottles)'].quantile(0.025) #Creating lower bound
upper_bound = data_transform['amount (in bottles)'].quantile(0.975) #Creating upper bound


filtered_df = data_transform[(data_transform['amount (in bottles)'] >= lower_bound) & \
                             (data_transform['amount (in bottles)'] <= upper_bound)] #Filter data based on upper & lower 
#bound

#Splitting independent and dependent variables
X = filtered_df.drop('amount (in bottles)', axis=1)
y = filtered_df['amount (in bottles)']

Lasso CV after removing outliers

In [12]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

reg = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
LassoCV()

reg.fit(X, y)

print('Best alpha using built-in LassoCV: %f' % reg.alpha_)
print('R2: %f' % reg.score(X,y))
print('Adjusted R2: %f' % (1 - (1-reg.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)))
y_predicted = reg.predict(X)
print('RMSE: %f' % (mean_squared_error(y, y_predicted, squared=False)))
print('y intercept of the model: %f' % reg.intercept_)
coef = pd.Series(reg.coef_, index = X.columns)
print('Lasso picked ' + str(sum(coef != 0)) + ' variables and eliminated the other ' + str(sum(coef == 0)) + ' variables')
coef = coef[coef !=0]
imp_coef = coef.sort_values(ascending = False)
print(imp_coef)

Best alpha using built-in LassoCV: 0.000000
R2: 0.539756
Adjusted R2: 0.539407
RMSE: 15.569090
y intercept of the model: 36.864703
Lasso picked 95 variables and eliminated the other 2 variables
Age_18-24                                   14.183544
Outlet Type_VALUE CHINESE                   13.767375
Salary_1800+                                13.390427
Nationality_Indonesian                      12.854831
Outlet Group_BUKIT PANJANG                  11.174700
                                              ...    
Outlet Group_CHANGI VILLAGE                -11.441014
Outlet Group_QUEENSTOWN, TANGLIN HALT      -12.087011
Outlet Group_KING ALBERT PK, CLEMENTI RD   -13.664828
Brand_Crystal                              -26.537378
Brand_Guinness                             -28.966816
Length: 95, dtype: float64


Poisson Regressor after removing outliers

In [13]:
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
}

poisson_best_param = PoissonRegressor()

grid_search = GridSearchCV(poisson_best_param, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X, y)

best_alpha = grid_search.best_params_['alpha']

poisson_model = PoissonRegressor(alpha=best_alpha)
poisson_model.fit(X, y)

predictions = poisson_model.predict(X)

print('R2: %f' % poisson_model.score(X,y))
print('Adjusted R2: %f' % (1 - (1-poisson_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)))
print('RMSE: %f' % (mean_squared_error(y, predictions, squared=False)))
print('y intercept of the model: %f' % poisson_model.intercept_)

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': poisson_model.coef_})
coefficients

R2: 0.607280
Adjusted R2: 0.606983
RMSE: 15.556474
y intercept of the model: 3.023477


,Feature,Coefficient
0,Brand_Crystal,-0.568715
1,Brand_Guinness,-0.862353
2,Brand_Heineken,0.737227
3,Brand_Tiger,0.693843
4,Nationality_Cambodian,0.009515
...,...,...
92,Tenure_0-2,0.059842
93,Tenure_3-5,0.005184
94,Tenure_6-10,-0.029443
95,Tenure_11-15,-0.044790
